In [ ]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [1]:
import subprocess
from pathlib import Path
from collections import Counter

In [ ]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        '--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [2]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    '--api-base-url', 'https://attractions-intersection-char-regulations.trycloudflare.com/v1',
    '--api-key', 'xxx',
    '--model-name', 'sam-paech/gemma-3-4b-it-antislop-exp35',    
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',        
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.jsonl',
    '--top-n-slop-phrases', '500',    
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "unsloth/gemma-3-4b-it",
    '--ftpo-pairs-jsonl', 'ftpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-base-url https://attractions-intersection-char-regulations.trycloudflare.com/v1 --api-key xxx --model-name sam-paech/gemma-3-4b-it-antislop-exp35 --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --slop-phrases-file banlists/slop_phrases.jsonl --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id unsloth/gemma-3-4b-it --ftpo-pairs-jsonl ftpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 39/177477 [00:00<00:04, 38085.65prompt/s]
Preparing to process 40 new prompts in this run.
Batch Generating:   0%|          | 0/40 [00:00<?, ?prompt/s]start prefix builder
built in 1.2858128547668457
Batch Generatin

Token indices sequence length is longer than the specified maximum sequence length for this model (1340 > 256). Running this sequence through the model will result in indexing errors


<|user|>
Writing prompt: Write a short story where the first sentence has 20 words, 2nd sentence has 19, 3rd has 18 etc. Story ends with a single word.

Write 1000 words to
<|assistant|>
The antique music box, a heavy, inlaid mahogany treasure, suddenly began to play a hauntingly familiar, minor key. 
A chill snaked around my ankles, and a sudden, inexplicable sadness filled the old study as the aged gears turned. 
The delicate porcelain ballerina inside, dressed in faded rose silk, began to spin with a melancholic, lonely grace. 
The miniature stage, painted with an impossibly vibrant autumn forest, reflected the gathering gloom within the shadowed room. 
It had been my grandmother’s, she’d left it to me with a cryptic warning – "Listen with your soul, my dear.” 
I hadn't truly *listened* with my soul until now, pulling it from the cedar chest, coated in decades of silent grief. 
The waltz it played was an ancient thing, a forgotten lament from a time I only knew through half-remember

CompletedProcess(args=['python3', 'main.py', '--api-base-url', 'https://attractions-intersection-char-regulations.trycloudflare.com/v1', '--api-key', 'xxx', '--model-name', 'sam-paech/gemma-3-4b-it-antislop-exp35', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '40', '--max-prompts', '40', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.jsonl', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '2000', '--request-mode', 'chunk', '--chunk-size', '20', '--chat-template-model-id', 'unsloth/gemma-3-4b-it', '--ftpo-pairs-jsonl', 'ftpo-pairs.jsonl'], returncode=0)